In [ ]:
using Plots,StatsBase, ImageFiltering, LinearAlgebra

# LASSO

On veut reconstruire le vecteur $x$ par la méthode du LASSO:
$$ \min_x \lambda\|x\|_1 +\frac{1}{2} \|Ax-y\|^2$$

On prendra d'abord une matrice d'observation $A$ aléatoire:

In [ ]:
# Observation: matrice aléatoire
m=30 # nombre de mesures
n=100 # taille de l'inconnue
A=randn(m,n)
σ=0.0
b=σ*randn(m)/sqrt(m);


## Signal parcimonieux
On considère d'abord $x_0$ un signal $k$-parcimonieux, c'est-à-dire avec au plus $k$ composantes non nulles.

In [ ]:
x0=zeros(n)
k=5 # nombre de composantes non nulles
I=1:n
J=sample(I,k;replace=false)
x0[J].=randn(k)
x0./=norm(x0) # on le normalise pour avoir la norme euclidienne 1

# Observation
y=A*x0+b

plot(x0,line=:stem,marker=:circle,label="Signal original")

Nous allons utiliser l'algorithme ISTA pour minimiser cette fonctionnelle.
$$					x^{(k+1)} = S_{\lambda \tau} \left(x^{(k)} - \tau A^*(Ax^{(k)}-y)\right).$$
où 
$$ S_{\lambda\tau}(z_i) =
	\begin{cases}
		z_i-\lambda \tau & \mbox{si } z_i>\lambda \tau\\
		0 & \mbox{si } -\lambda\tau\leq z_i\leq \lambda \tau\\
		z_i+\lambda \tau & \mbox{si } z_i<-\lambda \tau
	\end{cases}$$
   
On peut montrer que cet algorithme converge pour $0<\tau<\frac{2}{(\sigma_1(A))^2}$
    

In [ ]:
# Algorithme ISTA (Iterated Soft Thresholding Algorithm)
"""
    fwbackward(A,λ,τ,niter,xinit=zeros(size(A,2)))
Applique l'algorithme ISTA avec l'opérateur A donné comme une matrice.
On cherche à résoudre
min λ||x||_1 +1/2 |Ax-y|^2

tau : pas de la descente de gradient (forward)
niter: nombre d'itérations
"""
function fwbackward(A,y,λ,τ,niter,xinit=zeros(size(A,2)))
    x=copy(xinit)
    for k=1:niter
        x= soft_thresh.(x-τ*A'*(A*x-y),λ*τ)
    end
    return x
end

function soft_thresh(c,seuil)
    return (sign(c)*max(abs(c)-seuil,0.)) 
end    

1. Compléter ci-dessous les paramètres de l'algorithme. Essayer des reconstructions avec différents niveaux de bruit $\sigma$ et différents paramètres de régularisation $\lambda$.

In [ ]:
normA= opnorm(A,2) # Plus grande valeur singulière
#tau= # A COMPLETER
λ=1.0
#Calcul de la norme d'operateur
xrec=fwbackward(A,y,λ,tau,10000)

# erreur relative
print("Erreur relative: ")
println(norm(xrec-x0)/norm(x0))

In [ ]:
plot(x0,line=:stem,marker=:circle,label="Signal original")
plot!(xrec,line=:stem,marker=:circle,label="Reconstruction")
#plot!(xrecbis,line=:stem,marker=:circle,label="Reconstruction")

## Vecteur non parcimonieux

On souhaite mainenant comparer la performanace de reconstruction avec un vecteur qui n'est pas du tout parcimonieux.

2. Appliquer l'algorithme de forward-backward pour effectuer la reconstruction d'un signal non parcimonieux. Commenter.

In [ ]:
x0bis=randn(n)
x0bis./=norm(x0bis) # on le normalise pour avoir la norme euclidienne 1

# Observation (on garde le même bruit)
ybis=A*x0bis+b

plot(x0bis,line=:stem,marker=:circle,label="Signal original")

In [ ]:
# xrecbis= # A COMPLETER

# erreur relative
print("Erreur relative: ")
println(norm(xrecbis-x0bis)/norm(x0bis))

In [ ]:
plot(x0bis,line=:stem,marker=:circle,label="Signal original")
plot!(xrecbis,line=:stem,marker=:circle,label="Reconstruction")

## Problème de déconvolution

On va maintenant changer d'opérateur $A$.
On veut résoudre le problème de déconvolution
$$y=Ax+b \quad \mbox{où} \quad(Ax)(s)=\int h(t)x(s-t) dt  $$
et $h$ est la réponse impulsionnelle et $b$ est un bruit. Typiquement $h$ est un flou gaussien: $h(t)=e^{-\frac{t^2}{2\sigma^2}}$.

Plutôt que d'écrire la matrice de la convolution, on va donner des fonctions qui appliquent $A$ et $A^\top$ à un vecteur.


In [ ]:
largeur=1.0 # largeur de la gaussienne
h = ImageFiltering.Kernel.gaussian((largeur,))


"""
    function applyA!(z,x)
Applique l'opérateur A (convolution) au vecteur x et le stocke dans le vecteur z

ATTENTION: on effectue juste un corrélation (on suppose que h est symétrique)

ATTENTION; cette fonction modifie son premier argument
"""
function applyA!(z,x)
    global h
    imfilter!(z,x, reflect(h),"circular") # Filtrage avec cdt de periodicite
end

function applyA(x)
    z=zeros(size(x))
    applyA!(z,x)
    return z
end


"""
    function applyAstar!(z,x)
Applique l'opérateur A (convolution) au vecteur x et le stocke dans le vecteur z

ATTENTION: on effectue juste un corrélation (on suppose que h est symétrique)
"""
function applyAstar!(vec,z)
    global h
    imfilter!(vec,z, h,"circular") # Filtrage avec cdt de periodicite
    #global A
    #vec.=A'*z
end

function applyAstar(x)
    z=zeros(size(x))
    applyAstar!(z,x)
    return z
end

"""
    fwbackward(applyA!,applyAstar!,λ,τ,niter,xinit=zeros(size(A,2)))
Applique l'algorithme ISTA avec l'opérateur A donné à travers les fonctions applyA et applyAstar.
On cherche à résoudre
min λ||x||_1 +1/2 |Ax-y|^2

tau : pas de la descente de gradient (forward)
niter: nombre d'itérations
"""
function fwbackward(applyA!,applyAstar!,y,λ,τ,niter,xinit)
    x=copy(xinit)
    z=zeros(size(y))
    vec=zeros(size(xinit))
    
    for k=1:niter
        applyA!(z,x) # z=Ax
        z.-=y # z=Ax-y
        applyAstar!(vec,z) # vec=A'*(A*x-y)
        x.= x .-τ*vec
        x.=soft_thresh.(x,λ*τ)
    end
    return x
end
    
 

In [ ]:
# Observation
m=100 # nombre de mesures
n=100 # taille de l'inconnue (on ne fait pas de sous-échantillonnage, pour simplifier)
σ=0.0
b=σ*randn(m)/sqrt(m);


x0=zeros(n)
x0[1]=1
k=5 # nombre de composantes non nulles
I=1:n
J=sample(I,k;replace=false)
x0[J].=randn(k)
x0./=norm(x0) # on le normalise pour avoir la norme euclidienne 1

# Observation
y=applyA(x0)+b

plot(x0,line=:stem,marker=:circle,label="Signal original")
plot!(y,line=:stem,marker=:circle,label="Observation")

3. On peut montrer que la norme d'un opérateur de convolution est égale à $\|h\|_1$. Compléter les paramètres de l'algorithme.

In [ ]:
#normA=  # A COMPLETER
#tau= # A COMPLETER
λ=0.01

xrec=fwbackward(applyA!,applyAstar!,y,λ,tau,10000,zeros(n))
#xrec2=fwbackward(A,y,λ,tau,10000)

In [ ]:
plot(x0,line=:stem,marker=:circle,label="Signal original")
plot!(xrec,line=:stem,marker=:circle,label="Reconstruction")


3. Faire différents essais avec différents niveaux de bruit, différentes largeurs de gaussienne.

## Reconstruction des images
On suppose maintenant qu'on veut retrouver des sources ponctuelles lumineuses.

In [ ]:
n=100
img=zeros(n,n)
k=5 # nombre de composantes non nulles
I=1:n
Jx=sample(I,k;replace=false)
Jy=sample(I,k;replace=false)
for (k,l) in zip(Jx,Jy)
    img[k,l]=0.5*(1+rand())
end
img./=norm(img)
plot(Gray.(img))

In [ ]:
# Opérateurs d'observation
largeur=3.0 # largeur de la gaussienne
h = ImageFiltering.Kernel.gaussian(largeur)

"""
    function applyA!(z,x)
Applique l'opérateur A (convolution) au vecteur x et le stocke dans le vecteur z

ATTENTION: on effectue juste un corrélation (on suppose que h est symétrique)

ATTENTION; cette fonction modifie son premier argument
"""
function applyA!(z,x)
    global h
    imfilter!(z,x, reflect(h),"reflect") # Filtrage avec zero-padding
end

function applyA(x)
    z=zeros(size(x))
    applyA!(z,x)
    return z
end


"""
    function applyAstar!(z,x)
Applique l'opérateur A (convolution) au vecteur x et le stocke dans le vecteur z

ATTENTION: on effectue juste un corrélation (on suppose que h est symétrique)
"""
function applyAstar!(vec,z)
    global h
    imfilter!(vec,z, h,"reflect") # Filtrage avec cdt de periodicite
    #global A
    #vec.=A'*z
end

function applyAstar(x)
    z=zeros(size(x))
    applyAstar!(z,x)
    return z
end

function soft_thresh(c,seuil)
    return (sign(c)*max(abs(c)-seuil,0.)) 
end  

"""
    fwbackward(applyA!,applyAstar!,λ,τ,niter,xinit=zeros(size(A,2)))
Applique l'algorithme ISTA avec l'opérateur A donné à travers les fonctions applyA et applyAstar.
On cherche à résoudre
min λ||x||_1 +1/2 |Ax-y|^2

tau : pas de la descente de gradient (forward)
niter: nombre d'itérations
"""
function fwbackward(applyA!,applyAstar!,y,λ,τ,niter,xinit)
    x=copy(xinit)
    z=zeros(size(y))
    vec=zeros(size(xinit))
    
    for k=1:niter
        applyA!(z,x) # z=Ax
        z.-=y # z=Ax-y
        applyAstar!(vec,z) # vec=A'*(A*x-y)
        x.= x .-τ*vec
        x.=soft_thresh.(x,λ*τ)
    end
    return x
end
    


In [ ]:
# Observation
m=n
σ=0.0
b=σ*randn(m,m)/m;
y=applyA(img)+b

plot(Gray.(y./maximum(y)),title="Observation")

In [ ]:
# Reconstruction
normA= sum(h) 
#tau= # A COMPLETER
λ=0.001

xrec=fwbackward(applyA!,applyAstar!,y,λ,tau,10000,zeros(n,n))
display(plot(Gray.(img),title="Inconnue"))
display(plot(Gray.(xrec),title="Reconstruction"))